Try plotting some things from the S-MODE real-time data server (smode.whoi.edu)


In [10]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import datetime as dt



Scraps from SST_map_SPURS_Norteks_v2.py for reading from THREDDS server
```
#####################################
# This is AVHRR from VIIRS NPP, not MODIS
# This takes a long time:
url = 'https://thredds.jpl.nasa.gov/thredds/dodsC/OceanTemperature/VIIRS_NPP-OSPO-L3U-v2.61.nc'
ds_npp = xr.open_dataset(url)
ds_sub = ds_npp.sea_surface_temperature.sel(time=slice('20120929','20120930'),lat=slice(28,23),lon=slice(-42,-34))-273.15

ff = ~np.isnan(ds_sub.sel(lat=slice(24.55, 24.45), lon=slice(-38.05, -37.95)).mean('lon').mean('lat'))
ff2 = np.where(ff)
fig = plt.figure(figsize=(8, 4))
plt.plot(ff)
plt.title('Indices of subset with non-nan data')

fig = plt.figure(figsize=(8, 4))
#ds_sub.mean(['time']).plot(cmap='coolwarm',levels=np.linspace(26,28,15))
ds_sub.isel(time=ff2[0][1]).plot(cmap='coolwarm',levels=np.linspace(26,28,15))
plt.plot(SPURSlon,SPURSlat,'o',color='k')

fig= plt.figure(figsize=(8,4))
ds_sub.isel(time=ff2[0][0]).plot(cmap='coolwarm',levels=np.linspace(26,28,15))
plt.plot(SPURSlon,SPURSlat,'o',color='k')

fig= plt.figure(figsize=(8,4))
ds_sub.isel(time=ff2[0][2]).plot(cmap='coolwarm',levels=np.linspace(26,28,15))
plt.plot(SPURSlon,SPURSlat,'o',color='k')


sst_im = ds_sub.isel(time=ff2[0][1])

##############################################
fig= plt.figure(figsize=(8,4))
ds_sub.isel(time=ff2[0][1]).plot(cmap='coolwarm',levels=np.linspace(27.15,27.9,15))
#plt.plot(SPURSlon,SPURSlat,'o',color='k')
plt.axis([-38.708669354838705,  -37.26713709677419,  24.239516041550388,  25.32612009257010])
gpsdata = pd.read_csv('../buoy_and_glider_lat_lon.csv')
gpsdata['date_time'] = [tt.matlab2datetime(tval) for tval in gpsdata['mday']]
gpssub = gpsdata.loc[gpsdata['date_time'] >= '201209300000']
gpssub = gpssub.loc[gpssub['date_time'] <= '201209300600']
plt.plot(gpssub['buoy-lon'], gpssub['buoy-lat'], color='k')
plt.plot(gpssub['gldr-lon'], gpssub['gldr-lat'], color='m')
plt.plot(gpssub.iloc[-1]['buoy-lon'], gpssub.iloc[-1]['buoy-lat'], 'o', color='k')
plt.plot(gpssub.iloc[-1]['gldr-lon'], gpssub.iloc[-1]['gldr-lat'], 'o', color='m')
plt.axis('scaled')

plt.savefig(__figdir__ + "/VIIRS_NPP_SST.png",**savefig_args,dpi=600)
```

The code above isn't working great (specifically the xr.open_dataset line).  I had another good way in this script:
C:\Users\jtomf\Documents\Python\aviso_along_track\along_track_batch_process.py

Here is the entirety of that script:
```
import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime

#Define path using the r prefix (which means raw string so that special character / should not be evaluated)
path_dir: str = r"C:\D_drive\aviso_along_track\2020_CMEMS\dataset-duacs-rep-global-j1-phy-l3"

# Get a list of all .nc files available in different folders
filenames = glob.glob(path_dir+r"\*\*\*.nc")

dsmerged = xr.open_mfdataset(filenames,combine='nested',concat_dim='time')

time1 = dsmerged.time[0]
time2 = np.datetime64('2002-04-26T00:00:01')
dt = (time2-time1)# / np.timedelta64(1,'D')

timeall = dsmerged.time
foo=timeall<(time1.data+dt)

a = np.squeeze(np.argwhere(foo.data==True))


#Plot tracks
plt.plot(dsmerged.longitude[a],dsmerged.latitude[a],'.')
plt.show()

```


OK, that's not working great.  Try this:
https://rabernat.github.io/research_computing_2018/xarray-tips-and-tricks.html

In [ ]:
url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPB/AVHRR_METOPB_20210930T060000Z.nc'

files = [f'{base_url}.{year}.nc' for year in range(1948, 2019)]
files

Or maybe this:
    https://stackoverflow.com/questions/63815790/open-multiple-files-from-urls-using-xarray-and-open-mfdataset

In [45]:
df = pd.read_html('http://esg.lasg.ac.cn/thredds/catalog/esgcet/180/CMIP6.HighResMIP.CAS.FGOALS-f3-H.highresSST-future.r1i1p1f1.6hrPlevPt.psl.gr.v20200521.html#CMIP6.HighResMIP.CAS.FGOALS-f3-H.highresSST-future.r1i1p1f1.6hrPlevPt.psl.gr.v20200521', skiprows = 1)

df = df[0]

#get all relevant (432 files)
df = df[:432]

#add the url to each datafile to create a downloadable link for each
df['url'] = 'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/' + df['CMIP6.HighResMIP.CAS.FGOALS-f3-H.highresSST-future.r1i1p1f1.6hrPlevPt.psl.gr'].astype(str)

filelist = df['url'].tolist()

#do the first 10 to see if it works (change the number)
test = filelist[:10]

#do the first 10 into a dataset
ds = xr.open_mfdataset(test)

OSError: [Errno -68] NetCDF: I/O failure: b'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/psl_6hrPlevPt_FGOALS-f3-H_highresSST-future_r1i1p1f1_gr_205012010000-205012311800.nc'

In [46]:
filelist


['http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/psl_6hrPlevPt_FGOALS-f3-H_highresSST-future_r1i1p1f1_gr_205012010000-205012311800.nc',
 'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/psl_6hrPlevPt_FGOALS-f3-H_highresSST-future_r1i1p1f1_gr_205011010000-205011301800.nc',
 'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/psl_6hrPlevPt_FGOALS-f3-H_highresSST-future_r1i1p1f1_gr_205010010000-205010311800.nc',
 'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-future/r1i1p1f1/6hrPlevPt/psl/gr/v20200521/psl_6hrPlevPt_FGOALS-f3-H_highresSST-future_r1i1p1f1_gr_205009010000-205009301800.nc',
 'http://esg.lasg.ac.cn/thredds/dodsC/esg_dataroot/CMIP6/HighResMIP/CAS/FGOALS-f3-H/highresSST-futur

In [42]:
import pandas as pd
# df = pd.read_html('http://esg.lasg.ac.cn/thredds/catalog/esgcet/180/CMIP6.HighResMIP.CAS.FGOALS-f3-H.highresSST-future.r1i1p1f1.6hrPlevPt.psl.gr.v20200521.html#CMIP6.HighResMIP.CAS.FGOALS-f3-H.highresSST-future.r1i1p1f1.6hrPlevPt.psl.gr.v20200521', skiprows = 1)
df = pd.read_html('http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html', skiprows = 1)

df = df[0]

#get all relevant (432 files)
df = df[:432]

#add the url to each datafile to create a downloadable link for each
df['url'] = 'http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html?dataset=models/VIIRS_NRT/' + df.VIIRS_NRT.astype(str)

filelist = df['url'].tolist()

#do the first 10 to see if it works (change the number)
test = filelist[:1]

#do the first 10 into a dataset
ds = xr.open_mfdataset(test)

OSError: [Errno -75] NetCDF: Malformed or unexpected Constraint: b'http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html?dataset=models/VIIRS_NRT/VIIRS_NRT_20210915T205001Z.nc'

In [43]:
test

['http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html?dataset=models/VIIRS_NRT/VIIRS_NRT_20210915T205001Z.nc']

In [18]:
filenames
print(path_dir+"/*.nc")

http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html?dataset=models/VIIRS_NRT/*.nc


In [7]:
# not good: url = 'https://thredds.jpl.nasa.gov/thredds/dodsC/OceanTemperature/VIIRS_NPP-OSPO-L3U-v2.61.nc'
# url = 'http://smode.whoi.edu:8080/thredds/catalog/models/VIIRS_NRT/catalog.html'
ds_npp = xr.open_dataset(url)



OSError: [Errno -68] NetCDF: I/O failure: b'https://thredds.jpl.nasa.gov/thredds/dodsC/OceanTemperature/VIIRS_NPP-OSPO-L3U-v2.61.nc'

In [ ]:
ds_sub = ds_npp.sea_surface_temperature.sel(time=slice('20210910','20210915'),lat=slice(40,30),lon=slice(-125,-115))-273.15

In [ ]:
ff = ~np.isnan(ds_sub.sel(lat=slice(24.55, 24.45), lon=slice(-38.05, -37.95)).mean('lon').mean('lat'))
ff2 = np.where(ff)
fig = plt.figure(figsize=(8, 4))
plt.plot(ff)
plt.title('Indices of subset with non-nan data')
